In [ ]:
import numpy as np
import cupy as cp
import time

# Função para calcular o prefix sum (Hillis-Steele) usando CUDA com cupy
def hillis_steele_cuda(arr):
    arr_gpu = cp.array(arr)  # Transferindo o array para a GPU
    steps = 0  # Contador de passos
    start_time = time.time()

    # Prefix sum paralelo com deslocamento
    step = 1
    while step < arr_gpu.size:
        # Realiza o somatório com o valor deslocado em 2^step
        shifted = cp.roll(arr_gpu, step)
        arr_gpu += shifted
        # Convert the NumPy slice to a CuPy array before assignment
        arr_gpu[:step] = cp.array(arr[:step])  # Manter os primeiros valores originais
        step *= 2
        steps += 1

    end_time = time.time()

    # Registrando tempo de execução e quantidade de trabalho
    execution_time = end_time - start_time
    work_done = steps * arr_gpu.size  # Estimativa do trabalho total realizado

    return arr_gpu.get(), execution_time, work_done, steps  # Retornando os dados para a CPU

# Tamanhos dos arrays para teste
sizes = [100, 1000, 10000, 100000, 1000000, 10000000]

# Teste para cada tamanho de array
for size in sizes:
    arr = np.random.randint(1, 100, size)  # Gerando um array aleatório
    result, exec_time, work, steps = hillis_steele_cuda(arr)

    print(f"Tamanho do Array: {size}")
    print(f"Tempo de Execução: {exec_time:.4f} segundos")
    print(f"Quantidade de Trabalho: {work}")
    print(f"Número de Etapas: {steps}")
    print("-" * 50)